# Rad sa nebalansiranim skupovima podataka

Za skup podataka kažemo da je `nebalansiran` ukoliko ima značajno više instanci jedne klase u odnosu na drugu klasu (ili klase). Na primer, u skupu svih bankarskih transakcija samo jedan mali procenat transakcija nije regularan i podleže zakonskim procedurama. Slično, među pacijentima jedne bolnice može postojati svega par pacijenata koji boluje od neke retke bolesti. U ovim slučajevima je nebalansiranost podataka prirodna za sam fenomen. Postoje i scenariji u kojima je nebalansiranost podataka posledica lošeg odabira uzorka. Na primer, u anketi je ispitivana većinski mlađa populacijia, a ne opšte mnjenje. 

Da bi se neutralisao i ispratio efekat nebalansiranosti u obučavanju modela, može se preduzeti niz aktivnosti.

###  Aktivnosti na skupu podataka 

**Eliminisanje instanci većinske klase (engl. under-sampling)**
Ukoliko je na raspolaganju dovoljan broj instanci manjinske klase, može se izdvojiti isto toliko instanci većinske klase i obučavati model na ovako dobijenom balansiranom skupu. Izbor instanci većinske klase može da bude nasumičan. 

**Generisanje novih instanci (engl. over-sampling)** 
Ukoliko nije na raspolaganju dovoljan broj instanci manjinske klase, može se probati sa generisanjem njenih novih instanci. Na primer, mogu se nasumično ponoviti neke instance, može se iskoristiti tehnika *boosting* kojom se generišu instance iz iste distribucije ili se može iskoristiti *SMOTE* algoritam. Više o ovim pristupima ćemo videti u nastavku.

<img src='assets/under_and_over_sampling.png'>

Oba pomenuta pristupa imaju svoje prednosti i mane. Eliminisanjem instaci može doći do gubitka važnih informacija, dok se generisanjim novih instaci povećava rizik od preprilagođavanja. U oba slučaja nasumičan izbor može biti pristrasan i može uticati na performanse klasifikatora. 

**Ansambli**
U ovom pristupu se teži da se iskoriste svi raspoloživi podaci. Koriste se sve instance manjinske klase i veći broj skupova iste kardinalnosti instanci većinske klase. Na primer, ako imamo 400 instanci manjinske klase i 2000 instanci većinske klase, možemo napraviti 5 skupova podataka sa po 400 instanci manjinske klase i 400 instanci većinske klase. Na svakom od ovih skupova se može naučiti poseban model, a kao konačna ocena se može uzeti npr. prosek vrednosti. Dalje se u praksi ansambl koristi tako što se pokrenu svi modeli i iskoristiti npr. princip većinskog glasanja za odlučivanje o finalnom obeležju nove instance. 

<img src='assets/resampling_ensamble.png' style='height: 500px;'>
U ovim pristupima se može probati i sa različitim razmerama pozitivne i negativne klase npr. 1:2, 1:3, 2:1 i slično. 

**Klasterovanje većinske klase**
Ovaj pristup se zasniva na minimizaciji gubitka informacija o većinskoj klasi prilikom eliminacije instanci. Većinska klasa se može podeliti u `k` klastera gde `k` predstavlja kardinalnost manjinske klase. Zatim se na osnovu centroida ovako dobijenih klastera može generisati novi skup koji predstavlja većinsku klasu i koji se dalje može koristiti u obučavanju klasifikatora. 

### Adaptacija algoritama
Postoji klasa `cost-sensitive` klasifikatora koji su prilagođeni radu sa nebalansiranim podacima tako što kroz specifične funkcije gubitka navodi na poželjno ponašanje klasifikatora. Na primer, mogu se više kažnjavati pogrešne klasifikacije instanci manjinskih klasa nego pogrešne klasifikacije instanci većinskih klasa. Jedan takav klasifikator je `XGBoost` i njega ćemo upoznati nešto kasnije na krusu. 

### Evaluacija

Za evaluciju modela koji su trenirani na nebalansiranim skupovima podataka nije poželjno koristiti tačnost. Na primer, ovakvi modeli mogu ostvariti visoku tačnost predviđajući uvek obeležja brojnije klase. Poželjnije je koristiti preciznost, odziv, F1-meru, ali i `AUC` meru. 

Kao što smo videli, mere poput preciznosti, odziva ili F1-mere se računaju na osnovu matrice konfuzije i broja tačno klasifikovanih  (TP i TN) i pogrešno klasifikovanih (FP i FN) instanci. Da li je neka instanca dobro klasifikovana ili ne određivali smo na osnovu vrednosti funkcije cilja i praga klasifikacije. U primerima koje smo do sada viđali vrednost praga klasifikacije je bila 0.5, ali se u opštem slučaju za različite potrebe mogu koristiti i druge vrednosti. Ako je za uočenu instancu vrednost funkcije cilja bila veća od praga, pridruživali smo joj obeležje 1, dok smo joj u suprotnom pridruživali vrednost 0. `ROC (Receiver Operating Characteristic)` je kriva koja predstavlja vezu između udela lažno pozitivnih (engl. false positive rate, FPR) instanci i udela pravih pozitivnih (engl. true positive rate, TPR) instanci za zadati test skup i vrednosti pragova koje se variraju. Podsetimo se, udeo lažno pozitivnih instanci je količnik broja lažno pozitivnih instanci i broja svih negativnih instanci, dok je udeo pravih pozitivnih instanci količnik broja tačno klasifikovanih pozitivnih instanci i broja svih pozitivnih instanci. Udeli se izračunavaju za različite vrednosti praga klasifikacije koji se bira iz intervala [0, 1]. 

$$FPR = \frac{FP}{FP + TN} \hspace{0.5cm} TPR = \frac{TP}{TP + FN}$$

Često se za udeo pravih pozitivnih instanci koristi i termin `senzitivnost` (engl. sensitivity), a za udeo pravih negativnih instanci termin `specifičnost` (engl. specificity), pa se govori o krivoj koja povezuje senzitivnost i specifičnost.

<img src='assets/ROC_curves_1.png'>

Za svaku vrednost praga klasifikacije definisana je jedna tačka na ROC krivoj. Variranjem praga od maksimalne vrednosti do minimalne vrednosti (nad datim instancama) dobija se kriva koja spaja koordinatni početak sa tačkom (1, 1). Očekivano, sa spuštanjem praga povećava se broj instanci koje se klasifikuju kao pozitivne, a samim tim rastu i udeli stvarno pozitivnih i lažno pozitivnih instanci.

Karakteristika dobrih modela je da TPR brže raste od FPR, odnosno da se za male vrednosti FPR-a mogu dobiti visoke vrednosti TPR-a. To znači da za pogodan odabir praga ovaj model prilikom predviđanja pozitivne klase puno pogađa a malo greši, a samim tim i pri predviđanju negativne klase. Dakle, model je utoliko bolji što mu ROC kriva ima izvijeniji oblik u smeru ka tački $(0, 1)$ koja odgovara savršenom klasifikatoru. Sa druge strane, random klasifikator (najlošiji model) očekivano podjednako pogađa koliko i greši prilikom predikcija (TPR = FPR) nezavisno od postavljenog praga, tako da njemu odgovara ROC kriva sa tačkama po dijagonali od koordinatnog početka do $(1, 1)$.

<img src="assets/ROC_curve.png" width=300>

<img src='assets/ROC_curves_2.png'>

`AUC (Area Under the ROC Curve)` je mera koja predstavlja površinu ispod ROC krive. Interpretiramo je kao količinu  preciznosti koju treba žrtvovati da bi se ostvario visok odziv. U slučaju modela sa slabim performansama, ova vrednost teži broju 0.5 (površina trougla sa sporednom dijagonalom kao hipotenuzom). U slučaju modela sa dobrim performansama, ova vrednost teži broju 1 (površina celog kvadrata). Tako je vrednost AUC mere uvek u intervalu $[0.5, 1]$ gde vrednost $0.5$ odgovara najlošijem (random) klasifikatoru, a vrednost $1$ savršenom klasifikatoru. 

Predlažemo da pogledate kratak video i istražite vizuelizaciju dostupnu u okviru pregleda na [ovoj](https://www.dataschool.io/roc-curves-and-auc-explained/) adresi. 

### Zadatak

Potrebno je napraviti klasifikator koji razlikuje regularne i neregularne bankarske transakcije. 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

<p class='bg-info' style='padding: 20px'>
    Biblioteka <a href='https://imbalanced-learn.org'> imbalanced-learn </a> je Python biblioteka koja pruža podršku za rad sa nebalansiranim skupovima podataka. Stavlja na raspolaganje sve prethodno pomenute tehnike, ali i mnoge druge. Koristićemo je u daljem radu pa ju je potrebno instalirati komandom <b> conda install -c conda-forge imbalanced-learn</b>. 
<br>
<br> 
Za više informacija pogledati i <a href='https://imbalanced-learn.org/stable/user_guide.html'> zvaničnu dokumentaciju </a> ovog paketa.
</p>

In [ ]:
import imblearn

### Učitavanje i priprema skupa podataka

Skup podataka sa kojim ćemo raditi je preuzet sa [Kaggle platforme](https://www.kaggle.com/mlg-ulb/creditcardfraud). Stoga je potrebno da se prvo preuzme odgovarajuća `creditcard.csv` datoteka. 

### Početni model logističke regresije

Početni model sa kojim ćemo se oprobati će biti model logističke regresije. On će nam koristiti kao referentni model (eng. baseline) sa kojim ćemo kasnije porediti ostale modele.

### Logistička regresija sa eliminaciom instanci

Dalje ćemo ispitati ponašanje modela logističke regresije kojem prethode transformacije skupa podataka.

Podsetimo se, polazna brojnost klasa je:

In [27]:
np.bincount(y_train)

array([190490,    330])

Instance većinske klase ćemo elminisati korišćenjem `RandomUnderSampler` metode `imblearn` paketa. Parametrom `ratio` se utiče na željeni odnos broja instanci manjinske (N_m) i većinske (N_M) klase nakon eliminacije.   

### Logistička regresija sa generisanjem novih instanci

Podsetimo se, polazna brojnost klasa je:

In [35]:
np.bincount(y_train)

array([190490,    330])

Instance manjinske klase ćemo umnožiti nasumičnim ponavljanjem uz korišćenje `RandomOverSampler` metode `imblearn` paketa. Parametrom `ratio` se utiče na željeni odnos broja instanci manjinske (N_m) i većinske (N_M) klase nakon umnožavanja.   

### SMOTE

[SMOTE (Syntetic Minority Over-sampling Technique)](https://arxiv.org/pdf/1106.1813.pdf) je jedan od najpoznatijih algoritama za generisanje sintetičkih instanci. Za svaku instancu manjinske klase posmatra se okolina k-najbližih suseda, takođe pripadnika manjinske klase. Duž pravaca koji spajaju uočenu instancu i njene susede (sve ili neke od njih u zavisnosti od potrebe) interpoliraju se nove instance. S obzirom da nema eskplicitnog ponavljanja instanci, ublažena je mogućnost preprilagođavanja. Nažalost, tehnika može uvesti dodatni šum među podacima ukoliko se u okolinama koje se posmatraju nalaze i instance većinske klase.  
<img src='assets/SMOTE.png'>

Podsetimo se, polazna brojnost klasa je:

In [43]:
np.bincount(y_train)

array([190490,    330])

I algoritam `SMOTE` kao jedan od parametara ima `ratio` kojim se utiče na željeni odnos broja instanci. Ukolliko se izostavi (i ovde i u drugim algoritmima) pokušava se sa generisanjem skupova iste kardinalnosti. Parametrom `k` se zadaje broj najblizih suseda koji se posmatraju

### Logistička regresija sa težinama

U osnovi ovog pristupa je ideja o pridruživanju različitih težina instancama različitih klasa. Veće vrednosti težina instanci ukazuju klasifikatoru da obrati *više pažnje* na njih. 

In [52]:
from sklearn import utils

Težine su podrazumevano obrnuto proproporcionalne kardinalnostima klasa. Metoda `compute_class_weight` paketa `utils` nam daje uvid u ove vrednosti. Prilikom poziva metode za tip težina je potrebno navesti `balanced`. 

Težine se mogu pridruživati instancama i proizvoljno zadavanjem odgovarajućeg rečnika za vrednost `class_weight` atributa: